# Informações

- nomes: Raphael Leite da Costa Nascimento e Adriel Dos Sandos Araujo Cabral
- matrículas: 20190038319 e 20190034839
- Disciplina: Visualização de Dados

# Importações

In [1]:
import os
import re
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np

# Montando o data frame princial

In [2]:
data_path = 'dados'
files = [os.path.join(data_path, file) for file in os.listdir(data_path)]
files

['dados\\bcim_2016_21_11_2018.gpkg',
 'dados\\datatran2007.csv',
 'dados\\datatran2008.csv',
 'dados\\datatran2009.csv',
 'dados\\datatran2010.csv',
 'dados\\datatran2011.csv',
 'dados\\datatran2012.csv',
 'dados\\datatran2015.csv',
 'dados\\datatran2016.csv',
 'dados\\datatran2017.csv',
 'dados\\datatran2018.csv',
 'dados\\datatran2020.csv']

Criando uma lista de data frames e depois unindo todos em um so

In [3]:
dataframes = []

for file in files:
    try:
        dataframes.append(pd.read_csv(file, delimiter=';'))
    except:
        dataframes.append(pd.read_csv(file, delimiter=';', encoding='latin-1'))

df_final = pd.concat(dataframes, ignore_index=True)
df_sem_duplicatas = df_final.drop_duplicates(subset='id')
df_sem_duplicatas['count'] = 1

ParserError: Error tokenizing data. C error: Expected 4 fields in line 85, saw 6


# Tratando os valores nulos 

Substituindo os valores nulos da coluna 'ano' pelo ano da coluna 'data_inversa'

In [ ]:
def get_ano(year):
    
    regex1 = r'([0-9]+)(-[0-9]+)(-[0-9]+)'
    regex2 = r'([0-9])+/([0-9])+/([0-9]+)'
    
    if '-' in year:
        return int(re.search(regex1, year).group(1))
    else:
        return int('20' + re.search(regex2, year).group(3))

In [ ]:
df_sem_duplicatas.loc[df_sem_duplicatas['ano'].isnull(), 'ano'] = df_sem_duplicatas.loc[df_sem_duplicatas['ano'].isnull(), 'data_inversa'].apply(get_ano)

In [ ]:
df_sem_duplicatas['ano'].unique()

In [ ]:
df_sem_duplicatas[['ano']].isnull().sum()

In [ ]:
df_sem_duplicatas.columns

In [ ]:
df_sem_duplicatas

In [ ]:
df_sem_duplicatas.groupby(['ano'])['count'].sum().reset_index()

# Graficos

In [ ]:

plt.style.use('seaborn')
my_blue = '#1e83af'
my_red = '#9c1416'
my_green = '#1a7512'
plt.figure(figsize=(12, 8))

In [ ]:
def addlabels(x,y):
    for i in range(len(x)):
        plt.text(i, y[i], y[i], ha = 'center')

In [ ]:
def grafico_simples(data: pd.core.frame.DataFrame, eixo_x: str, eixo_y: str, title:str) -> None:
    
    X, Y = data[eixo_x], data[eixo_y].values

    plt.title(title)
#     ax=plt.axes()
#     ax.set_facecolor('white')
    plt.bar(X, Y, color=my_blue, label=eixo_y, width=0.6)
    addlabels(X, Y)
    plt.legend(loc="upper right")
    plt.xlabel(eixo_x)
    plt.ylabel(eixo_y)
    plt.xticks(rotation=45)
    plt.show()

In [ ]:
df_sem_duplicatas.groupby(['ano'])['count'].sum().reset_index()

## Gráfico da queda dos acidentes 

In [ ]:
df_acidentes_count = df_sem_duplicatas.groupby(['ano'])['count'].sum().reset_index()


df_acidentes_count = df_acidentes_count.astype({'ano': str})
#df_acidentes_count['ano'] = pd.to_string(df_acidentes_count['ano'])

grafico_simples(df_acidentes_count, 'ano', 'count', 
                'Gráfico da queda dos acidentes')


## Gráfico de causas de acidente 2011

In [ ]:
df_graph = df_sem_duplicatas.loc[df_sem_duplicatas['ano'] == 2011].groupby('causa_acidente')['count'].sum().reset_index()
#grafico_simples(df_graph, 'causa_acidente', 'mortos', 'Gráfico de causas de acidente 2011')

plt.title('Causas de acidentes durante o ano de 2011')
plt.barh(df_graph['causa_acidente'], df_graph['count'].values, color=my_blue, label='mortos')
plt.legend(loc="upper right")
plt.xlabel('número de acidentes')
plt.ylabel('causa de acidentes')
plt.xticks(rotation=45)
plt.show()

In [ ]:
acidentes_by_uf = df_sem_duplicatas.groupby('uf')['count'].sum().reset_index()
acidentes_by_uf = acidentes_by_uf.sort_values('count', ascending=False).reset_index(drop=True)
acidentes_by_uf

### Os 5 estados que acontecem mais acidentes + PB

In [ ]:
grafico_simples(acidentes_by_uf.loc[acidentes_by_uf['uf'].isin(['MG', 'SC', 'PR', 'RJ', 'RS', 'PB'])], 'uf', 'count', 
                'Os 5 estados que acontecem mais acidentes + PB')

## Gráficos da Paraiba

In [ ]:
df_sem_duplicatas_PB = df_sem_duplicatas.loc[df_sem_duplicatas['uf'] == 'PB'].reset_index(drop=True)

In [ ]:
df_sem_duplicatas_PB['municipio'] = df_sem_duplicatas_PB['municipio'].map(lambda nome: nome.strip())

In [ ]:

df_pessoas = df_sem_duplicatas_PB.groupby(['ano'])['pessoas'].sum().reset_index()
df_mortos = df_sem_duplicatas_PB.groupby(['ano'])['mortos'].sum().reset_index()
df_feridos = df_sem_duplicatas_PB.groupby(['ano'])['feridos'].sum().reset_index()
df_ilesos = df_sem_duplicatas_PB.groupby(['ano'])['ilesos'].sum().reset_index()
df_ignorados = df_sem_duplicatas_PB.groupby(['ano'])['ignorados'].sum().reset_index()


anos = df_mortos['ano'].values
anos = [str(ano) for ano in anos]

In [ ]:
df_total_pessoas = pd.concat([df_pessoas, 
                              df_mortos.drop(columns=['ano']), 
                               df_ignorados.drop(columns=['ano']),
                              df_feridos.drop(columns=['ano']), 
                              df_ilesos.drop(columns=['ano'])], 1)

In [ ]:
df_total_pessoas

In [ ]:
plt.pie(df_total_pessoas.iloc[10, 2:], labels = df_total_pessoas.columns[2:], autopct='%.0f%%')
plt.show()

In [ ]:
df_total_pessoas_percent = df_total_pessoas.copy()

df_total_pessoas_percent['mortos'] = df_total_pessoas_percent['mortos'] / df_total_pessoas_percent['pessoas']
df_total_pessoas_percent['ilesos'] = df_total_pessoas_percent['ilesos'] / df_total_pessoas_percent['pessoas']
df_total_pessoas_percent['feridos'] = df_total_pessoas_percent['feridos'] / df_total_pessoas_percent['pessoas']
df_total_pessoas_percent['ignorados'] = df_total_pessoas_percent['ignorados'] / df_total_pessoas_percent['pessoas']

In [ ]:
df_total_pessoas_percent.index = df_total_pessoas_percent['ano']
df_total_pessoas_percent.drop(columns=['ano', 'pessoas'], inplace=True)
df_total_pessoas_percent

In [ ]:
df_total_pessoas_percent.plot.bar(stacked=True, width=0.7)
plt.legend(
    bbox_to_anchor=(0.5, 1.02),
    loc="lower center",
    borderaxespad=0,
    frameon=False,
    ncol=4,
)
for ix, row in df_total_pessoas_percent.reset_index(drop=True).iterrows():
    cumulative = 0
    for element in row:
        if element == element and element > 0.01:
            plt.text(
                ix,
                cumulative + element / 2,
                f"{np.round(element * 100, 1)}%",
                va="center",
                ha="center",
            )
        cumulative += element
plt.tight_layout()

In [ ]:
df_pessoas

In [ ]:
grafico_simples(df_pessoas, 'ano', 'pessoas', 
                'Quantidade de pessoas que sofreram acidente na PB entre 2007 e 2020')

### Número de mortos, feridos e ilesos a cada ano na Paraíba

In [ ]:
plt.title('Gráfico de y em função de x')
plt.plot(anos, df_mortos['mortos'].values, marker='o', color=my_blue, label="Mortos")
plt.plot(anos, df_feridos['feridos'].values, marker='*', color=my_red, label="Feridos")
plt.plot(anos, df_ilesos['ilesos'].values, marker='.', color=my_green, label="Ilesos")
plt.legend(loc="upper right")
plt.xlabel('Anos')
plt.ylabel('Status das vitimas')
plt.xticks(rotation=45)
plt.grid(True)
plt.show()

In [ ]:
df_sem_duplicatas_PB

In [ ]:
def split_time(time):
    
    time_int = int(''.join(time.split(':')[:2]))
    
    if 0 < time_int < 600:
        return '00-06'
    elif 600 < time_int < 800:
        return '06-08'
    elif 800 < time_int < 1200:
        return '08-12'
    elif 1200 < time_int < 1600:
        return '12-16'
    elif 1600 < time_int < 2000:
        return '16-20'
    elif 2000 < time_int < 2400:
        return '20-24'


In [ ]:
df_sem_duplicatas_PB['time_split'] = df_sem_duplicatas_PB['horario'].apply(lambda time: split_time(time), 1)

In [ ]:
df_sem_duplicatas_PB['time_split'].value_counts()

In [ ]:
import geopandas as gpd

In [ ]:
INFOS_UFS = gpd.read_file('dados/bcim_2016_21_11_2018.gpkg', layer='lim_unidade_federacao_a')

In [ ]:
INFOS_UFS.columns

In [ ]:
df_sem_duplicatas = df_sem_duplicatas.rename(columns={"state": "sigla"})

In [ ]:
df_sem_duplicatas_ufs = df_sem_duplicatas.groupby(['sigla'])['pessoas'].sum().reset_index()
df_sem_duplicatas_ufs = df_sem_duplicatas_ufs.iloc[1:, :].reset_index(drop=True)

In [ ]:
df_sem_duplicatas_ufs

In [ ]:
df_sem_duplicatas_merged = INFOS_UFS.merge(df_sem_duplicatas_ufs, 
                                           on='sigla', 
                                           how='left')

In [ ]:
df_sem_duplicatas_merged

In [ ]:
df_sem_duplicatas_merged.plot(column='pessoas',
                             cmap='Reds',
                             legend=True,
                             edgecolor='black')